# Post-Midterm Report: Back to the Drawing Board 


Feature engineering and testing different model specifications. Models in this document are mostly of the non-NN variety to provide insight on how the metadata we intend to utilize in our models can improve our metrics. There are cases in which tf-idf weighting/good feature engineering is sufficient for specifying a model that outperforms the state of the art. This is something we can explore in our project.

In [8]:
import numpy as np 
import pandas as pd

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble 

import xgboost, textblob, string 
from keras.preprocessing import text, sequence 
from keras import layers, models, optimizers

In [ ]:
# for reproducibility 
SEED = 1984
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True # for reproducible results 

### Loading data, standardizing variables 

In [2]:
# load data from already split train/validation/test sets 
train_df = pd.read_csv('us_news_train.csv')
validation_df = pd.read_csv('us_news_validation.csv')
test_df = pd.read_csv('us_news_test.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
# inspecting data 
train_df.head()
train_df.shape
train_df.dtypes
train_df['ideology'].value_counts()
train_df['outlet'].value_counts()

5     13095
2      9469
3      7029
7      6711
8      5244
4      4778
1      4726
17     4653
13     4237
16     2867
0      2666
18     2440
14     2439
11     2321
9      2114
10     1878
6      1419
15     1257
12      657
Name: outlet, dtype: int64

In [27]:
# changing variable names for validation set 
validation_df.columns = validation_df.columns.str.replace(r"validate.","")
validation_df.dtypes

outlet            int64
outlet_url       object
datetime         object
url_orig         object
headline         object
description      object
author           object
domain           object
topic_tags       object
text             object
section          object
news_keywords    object
subsection       object
paywall          object
provider         object
ideology          int64
dtype: object

### Label encoding and setting target variable `['ideology']`

In [28]:
# label encoding  
le = LabelEncoder() # creating labelencoder object 
# converting categorical target variable 'ideology' into numeric 
train_df['ideology'] = le.fit_transform(train_df['ideology'])
validation_df['ideology'] = le.fit_transform(validation_df['ideology'])
test_df['ideology'] = le.fit_transform(test_df['ideology'])
# converting other categorical columns into numeric 
train_df['outlet'] = le.fit_transform(train_df['outlet'])
validation_df['outlet'] = le.fit_transform(validation_df['outlet'])
test_df['outlet'] = le.fit_transform(test_df['outlet'])
# set target variable: ideology 
target = train_df['ideology']
target = validation_df['ideology']
target = test_df['ideology']

### Feature Engineering 

In [38]:
validation_df.dtypes

outlet            int64
outlet_url       object
datetime         object
url_orig         object
headline         object
description      object
author           object
domain           object
topic_tags       object
text             object
section          object
news_keywords    object
subsection       object
paywall          object
provider         object
ideology          int64
dtype: object

In [40]:
# converting dtype objects to unicode strings
train_df['headline','text','topic_tags','description'] = train_df['headline','text','topic_tags','description'].astype('unicode')
# validation_df['headline','text','topic_tags','description'] = validation_df['headline','text','topic_tags','description'].astype('unicode')

In [43]:
# cleaning text here?? 

#### Count Vectors as Features

In [42]:
# creating count vectorizer objects
count_vect = CountVectorizer(analyzer='word',token_pattern=r'w{1,}')
count_vect.fit(train_df['headline','text','topic_tags','description'])
#count_vect.fit(validation_df['headline','text','topic_tags','description'])
# transforming training and validation data using count vectorizer object 
xtrain_count = count_vect.transform(train_df)
#xtrain_count = count_vect.transform(validation_df)

AttributeError: 'tuple' object has no attribute 'lower'